In [1]:
!pip install -q "tensorflow==2.18.0"

In [2]:
!pip install pandas matplotlib jupyter tqdm torchdiffeq torchinfo scikit-learn
!pip install -q -U keras-tuner shap tf-explain tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
CSV_PATH = "/content/measures_v2.csv"

In [5]:
import json, numpy as np, pandas as pd, tensorflow as tf, keras_tuner as kt
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import callbacks

tf.config.set_visible_devices([], 'GPU')

tf.keras.utils.set_random_seed(42)   # reproducibility

# ---- modified ThermalNN definition ----
class ThermalNN(tf.keras.Model):
    def __init__(self, width=64, activation="relu"):
        super().__init__()
        self.g_net = tf.keras.Sequential(
            [tf.keras.layers.Dense(width, activation=activation),
             tf.keras.layers.Dense(1)])
        self.c_net = tf.keras.Sequential(
            [tf.keras.layers.Dense(width, activation=activation),
             tf.keras.layers.Dense(1)])
        self.p_net = tf.keras.Sequential(
            [tf.keras.layers.Dense(width, activation=activation),
             tf.keras.layers.Dense(1)])

    def call(self, x):
        g = self.g_net(x)
        c = self.c_net(x)
        p = self.p_net(x)
        return p / (g + 1e-7) + c

# ---- Helper to load ANY csv with target column 'pm' ----
def load_csv(path, val_split=0.2):
    df = pd.read_csv(path)
    x = df.drop(columns=["pm"]).astype("float32").values
    y = df["pm"].astype("float32").values
    idx = np.random.permutation(len(df))
    split = int(len(df)*(1-val_split))
    return (x[idx[:split]], y[idx[:split]]), (x[idx[split:]], y[idx[split:]])

# ---- Build function for Keras-Tuner ----
def build_tnn(hp):
    width  = hp.Int("width", 8, 128, 8)
    act    = hp.Choice("activation", ["relu", "elu"])
    lr     = hp.Float("lr", 1e-4, 1e-2, sampling="log")
    model  = ThermalNN(width, act)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                  loss="mse", metrics=["mse"])
    return model

def tune_and_train(csv, trials=20):
    (x_tr, y_tr), (x_val, y_val) = load_csv(csv)
    tuner = kt.RandomSearch(
        build_tnn, objective="val_mse", max_trials=trials,
        directory="tuner_logs", project_name="thermal_nn_rs")
    tuner.search(
        x_tr, y_tr, epochs=20, validation_data=(x_val, y_val),
        batch_size=kt.HyperParameters().Int("batch_size", 256, 1024, 256),
        callbacks=[callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1)
    best_hp = tuner.get_best_hyperparameters(1)[0]
    best_model = tuner.hypermodel.build(best_hp)
    best_model.fit(
        x_tr, y_tr, validation_data=(x_val, y_val), epochs=50,
        batch_size=best_hp.get("batch_size"),
        callbacks=[callbacks.EarlyStopping(patience=8, restore_best_weights=True)],
        verbose=1)
    val_mse = best_model.evaluate(x_val, y_val, verbose=0)[1]
    art = Path("artifacts"); art.mkdir(exist_ok=True)
    best_model.save(art / "tnn_best.keras")          # Keras v3 single-file format :contentReference[oaicite:3]{index=3}
    json.dump({"val_mse": float(val_mse)}, open(art / "metrics.json","w"), indent=2)
    print(f" Best val MSE = {val_mse:.3f} K²  (model & metrics in ./artifacts/)")

# ---- actually launch the tuner ----
tune_and_train(CSV_PATH, trials=10)


Trial 10 Complete [00h 05m 36s]
val_mse: 41.645992279052734

Best val_mse So Far: 23.788833618164062
Total elapsed time: 00h 55m 35s


KeyError: 'batch_size does not exist.'